## Training Notebook

This notebook illustrates training of a simple model to classify digits using the MNIST dataset. This code is used to train the model included with the templates. This is meant to be a starter model to show you how to set up Serverless applications to do inferences. For deeper understanding of how to train a good model for MNIST, we recommend literature from the [MNIST website](http://yann.lecun.com/exdb/mnist/). The dataset is made available under a [Creative Commons Attribution-Share Alike 3.0](https://creativecommons.org/licenses/by-sa/3.0/) license.

In [2]:
# We'll use scikit-learn to load the dataset

! pip install -q scikit-learn==0.23.2

In [25]:
# Load the mnist dataset

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

X, y = fetch_openml('mnist_784', return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10000)

## Tensorflow Model Training

For this example, we will train a simple CNN classifier using Tensorflow to classify the MNIST digits. We will then freeze the model in the `.h5` format. This is same as the starter model file included with the SAM templates.

In [27]:
! pip install -q tensorflow==2.8.0

In [28]:
import numpy as np
import tensorflow as tf

print (f'Using TesorFlow version {tf.__version__}')

# Reshape the flat input into a 28x28x1 dim tensor
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

# Convert the output tensors to integers (our data is read as Strings)
y_train = y_train.astype(np.int8)
y_test = y_test.astype(np.int8)

model = tf.keras.Sequential([
    # Input layer to match the shape above
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    
    # Output layer for 10 classes
    tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=15)

Using TesorFlow version 2.4.0
Epoch 1/15
1875/1875 [==============================] - 13s 6ms/step - loss: 0.4003 - accuracy: 0.8761
Epoch 2/15
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0878 - accuracy: 0.9731
Epoch 3/15
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0655 - accuracy: 0.9792
Epoch 4/15
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0538 - accuracy: 0.9840
Epoch 5/15
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0459 - accuracy: 0.9854
Epoch 6/15
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0377 - accuracy: 0.9882
Epoch 7/15
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0362 - accuracy: 0.9884
Epoch 8/15
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0292 - accuracy: 0.9909
Epoch 9/15
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0272 - accuracy: 0.9915
Epoch 10/15
1875/1875 [====================

In [29]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

print('\nTest accuracy:', test_acc)

313/313 - 1s - loss: 0.0404 - accuracy: 0.9909

Test accuracy: 0.9908999800682068


In [30]:
# Save model to the disk
model.save('tf_digit_classifier.h5')